In [1]:
import numpy as np
import sys
import scipy.stats
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *
from VMSfunctions.Controller import *

# Mass Spec test

In [6]:
import pickle
hmdb = pickle.load(open('../../../Compounds/hmdb_compounds.p','rb'))

In [94]:
ps = load_obj('../models/peak_sampler_4_beers.p')
xcms_output = '../models/beer_ms1_peaks.csv.gz'
chromatograms = ChromatogramCreator(xcms_output)

DEBUG:Chromatograms:Loading 0 chromatograms
../codes\VMSfunctions\Common.py:56: RuntimeWarning:

divide by zero encountered in double_scalars

DEBUG:Chromatograms:Loading 5000 chromatograms
DEBUG:Chromatograms:Loading 10000 chromatograms
DEBUG:Chromatograms:Loading 15000 chromatograms
DEBUG:Chromatograms:Loading 20000 chromatograms
DEBUG:Chromatograms:Loading 25000 chromatograms
DEBUG:Chromatograms:Loading 30000 chromatograms
DEBUG:Chromatograms:Loading 35000 chromatograms
DEBUG:Chromatograms:Loading 40000 chromatograms
DEBUG:Chromatograms:Loading 45000 chromatograms


In [95]:
chemicals = ChemicalCreator(ps, chromatograms)

# Improve Chemical Creator

In [100]:
min_ms1_intensity = 2E5
min_rt = 3*60
max_rt = 21*60
dataset = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 50, 3, "Known","Empirical", None, False, hmdb, 0.1)

DEBUG:Chemicals:50 ms1 peaks to be created.


Warning ms_level > 3 not implemented properly yet. Uses scaled ms_level = 2 information for now


DEBUG:Chemicals:i = 25 Total = 32
DEBUG:Chemicals:i = 25 Total = 33
DEBUG:Chemicals:i = 25 Total = 34
DEBUG:Chemicals:i = 50 Total = 62


# Test Top N Controller

In [113]:
max_rt = 10*60                  # the maximum retention time of scans to generate
N = 1                           # top-5 DDA fragmentation
mz_tol = 5                      # the mz isolation window around a selected precursor ion
rt_tol = 15                     # the rt window around a selected precursor ion to prevent it from fragmented multiple times
min_ms2_intensity = 5000        # the minimum ms2 peak intensity

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TopNController(mass_spec, N, mz_tol, rt_tol, min_ms2_intensity=min_ms2_intensity)
controller.run(max_rt)

INFO:TopNController:Acquisition open
INFO:TopNController:Received Scan 1524 num_peaks=6 rt=187.37 ms_level=1
DEBUG:TopNController:Isolated precursor ion 465.1038 window (465.1015, 465.1062)
DEBUG:TopNController:Dynamic exclusion from_mz 465.1015 to_mz 465.1062 from_rt 172.37 to_rt 202.37
INFO:TopNController:Received Scan 1525 num_peaks=10 rt=187.40 ms_level=2
INFO:TopNController:Received Scan 1526 num_peaks=6 rt=187.53 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 465.1039 rt 187.53
INFO:TopNController:Received Scan 1527 num_peaks=6 rt=187.65 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 465.1039 rt 187.65
INFO:TopNController:Received Scan 1528 num_peaks=6 rt=187.71 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 465.1039 rt 187.71
INFO:TopNController:Received Scan 1529 num_peaks=6 rt=187.79 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 465.1039 rt 187.79
INFO:TopNController:Received Scan 1530 num_peaks=6 rt=187.89 ms_level=1
DEBUG:TopNCont

DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.19
INFO:TopNController:Received Scan 1584 num_peaks=7 rt=194.34 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.34
INFO:TopNController:Received Scan 1585 num_peaks=7 rt=194.41 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.41
INFO:TopNController:Received Scan 1586 num_peaks=7 rt=194.55 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.55
INFO:TopNController:Received Scan 1587 num_peaks=7 rt=194.63 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.63
INFO:TopNController:Received Scan 1588 num_peaks=7 rt=194.69 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.69
INFO:TopNController:Received Scan 1589 num_peaks=7 rt=194.77 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 955.7760 rt 194.77
INFO:TopNController:Received Scan 1590 num_peaks=7 rt=194.95 ms_level=1
DEBUG:TopNController:Excluded prec

DEBUG:TopNController:Excluded precursor ion mz 820.5862 rt 202.18
INFO:TopNController:Received Scan 1643 num_peaks=14 rt=202.33 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 820.5862 rt 202.33
INFO:TopNController:Received Scan 1644 num_peaks=21 rt=202.46 ms_level=1
DEBUG:TopNController:Isolated precursor ion 889.8230 window (889.8185, 889.8274)
DEBUG:TopNController:Dynamic exclusion from_mz 889.8185 to_mz 889.8274 from_rt 187.46 to_rt 217.46
INFO:TopNController:Received Scan 1645 num_peaks=10 rt=202.63 ms_level=2
INFO:TopNController:Received Scan 1646 num_peaks=21 rt=202.89 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 202.89
INFO:TopNController:Received Scan 1647 num_peaks=21 rt=203.01 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 203.01
INFO:TopNController:Received Scan 1648 num_peaks=21 rt=203.18 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 203.18
INFO:TopNController:Received Scan 1649 num_peaks=21 

DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 210.53
INFO:TopNController:Received Scan 1702 num_peaks=14 rt=210.90 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 210.90
INFO:TopNController:Received Scan 1703 num_peaks=14 rt=211.08 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 211.08
INFO:TopNController:Received Scan 1704 num_peaks=14 rt=211.13 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 211.13
INFO:TopNController:Received Scan 1705 num_peaks=14 rt=211.21 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 211.21
INFO:TopNController:Received Scan 1706 num_peaks=14 rt=211.29 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 211.29
INFO:TopNController:Received Scan 1707 num_peaks=14 rt=211.35 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8229 rt 211.35
INFO:TopNController:Received Scan 1708 num_peaks=14 rt=211.64 ms_level=1
DEBUG:TopNController:Exclud

DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 218.92
INFO:TopNController:Received Scan 1761 num_peaks=7 rt=219.02 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 219.02
INFO:TopNController:Received Scan 1762 num_peaks=7 rt=219.15 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 219.15
INFO:TopNController:Received Scan 1763 num_peaks=7 rt=219.21 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 219.21
INFO:TopNController:Received Scan 1764 num_peaks=7 rt=219.34 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8230 rt 219.34
INFO:TopNController:Received Scan 1765 num_peaks=7 rt=219.39 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8231 rt 219.39
INFO:TopNController:Received Scan 1766 num_peaks=7 rt=219.46 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 889.8231 rt 219.46
INFO:TopNController:Received Scan 1767 num_peaks=7 rt=219.54 ms_level=1
DEBUG:TopNController:Excluded prec

...

INFO:TopNController:Received Scan 4443 num_peaks=6 rt=561.18 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0929 rt 561.18
INFO:TopNController:Received Scan 4444 num_peaks=6 rt=561.25 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0929 rt 561.25
INFO:TopNController:Received Scan 4445 num_peaks=6 rt=561.36 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0930 rt 561.36
INFO:TopNController:Received Scan 4446 num_peaks=6 rt=561.55 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0931 rt 561.55
INFO:TopNController:Received Scan 4447 num_peaks=6 rt=561.63 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 561.63
INFO:TopNController:Received Scan 4448 num_peaks=6 rt=561.73 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 561.73
INFO:TopNController:Received Scan 4449 num_peaks=6 rt=561.92 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0934 rt 561.92
INFO:TopNController:Received Scan 

DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.42
INFO:TopNController:Received Scan 4503 num_peaks=6 rt=568.51 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.51
INFO:TopNController:Received Scan 4504 num_peaks=6 rt=568.63 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.63
INFO:TopNController:Received Scan 4505 num_peaks=6 rt=568.72 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.72
INFO:TopNController:Received Scan 4506 num_peaks=6 rt=568.79 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.79
INFO:TopNController:Received Scan 4507 num_peaks=6 rt=568.88 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.88
INFO:TopNController:Received Scan 4508 num_peaks=6 rt=568.95 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0932 rt 568.95
INFO:TopNController:Received Scan 4509 num_peaks=6 rt=569.04 ms_level=1
DEBUG:TopNController:Excluded prec

INFO:TopNController:Received Scan 4562 num_peaks=6 rt=576.33 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0934 rt 576.33
INFO:TopNController:Received Scan 4563 num_peaks=6 rt=576.40 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0934 rt 576.40
INFO:TopNController:Received Scan 4564 num_peaks=6 rt=576.52 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0935 rt 576.52
INFO:TopNController:Received Scan 4565 num_peaks=6 rt=576.60 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0936 rt 576.60
INFO:TopNController:Received Scan 4566 num_peaks=6 rt=576.68 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0936 rt 576.68
INFO:TopNController:Received Scan 4567 num_peaks=6 rt=576.80 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0937 rt 576.80
INFO:TopNController:Received Scan 4568 num_peaks=6 rt=576.98 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0938 rt 576.98
INFO:TopNController:Received Scan 

DEBUG:TopNController:Excluded precursor ion mz 387.0935 rt 583.59
INFO:TopNController:Received Scan 4622 num_peaks=6 rt=583.65 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 387.0935 rt 583.65
INFO:TopNController:Received Scan 4692 num_peaks=6 rt=592.66 ms_level=1
DEBUG:TopNController:Isolated precursor ion 352.0713 window (352.0696, 352.0731)
DEBUG:TopNController:Dynamic exclusion from_mz 352.0696 to_mz 352.0731 from_rt 577.66 to_rt 607.66
INFO:TopNController:Received Scan 4693 num_peaks=10 rt=592.73 ms_level=2
INFO:TopNController:Received Scan 4694 num_peaks=6 rt=592.86 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 352.0714 rt 592.86
INFO:TopNController:Received Scan 4695 num_peaks=6 rt=592.92 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 352.0714 rt 592.92
INFO:TopNController:Received Scan 4696 num_peaks=6 rt=593.01 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 352.0714 rt 593.01
INFO:TopNController:Received Scan 4697 num_peaks=6 rt=593

INFO:TopNController:Received Scan 4750 num_peaks=6 rt=599.91 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 352.0714 rt 599.91
INFO:TopNController:Received Scan 4751 num_peaks=6 rt=599.97 ms_level=1
DEBUG:TopNController:Excluded precursor ion mz 352.0713 rt 599.97
INFO:TopNController:Acquisition closing


# DIA Mass Spec

In [114]:
dia_design = "basic"
window_type = "even"
kaufmann_design = None
extra_bins = 0
num_windows=10

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)
controller.run(max_rt)

INFO:TreeController:Acquisition open
INFO:TreeController:Received MS1 scan Scan 1475 num_peaks=6 rt=187.31 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1480 num_peaks=40 rt=188.14 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1486 num_peaks=6 rt=189.24 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1491 num_peaks=40 rt=190.02 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1497 num_peaks=6 rt=191.20 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]

DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1660 num_peaks=40 rt=217.58 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1662 num_peaks=7 rt=217.84 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 1671 num_peaks=40 rt=219.73 ms_level=2
INFO:TreeController:Received MS1 scan Scan 1673 num_peaks=7 rt=220.00 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0

INFO:TreeController:Received MS1 scan Scan 2262 num_peaks=7 rt=303.18 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 2270 num_peaks=10 rt=304.47 ms_level=2
INFO:TreeController:Received MS2 scan Scan 2271 num_peaks=30 rt=304.74 ms_level=2
INFO:TreeController:Received MS1 scan Scan 2273 num_peaks=7 rt=305.01 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 2281 num_peaks=20 rt=306.05 ms_level=2
INFO:TreeController:Received MS2 scan Scan 2282 num_peaks=60 rt=306.38 ms_level=2
INFO:TreeController:Received M

INFO:TreeController:Received MS2 scan Scan 2592 num_peaks=40 rt=353.82 ms_level=2
INFO:TreeController:Received MS1 scan Scan 2595 num_peaks=7 rt=354.38 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS1 scan Scan 2840 num_peaks=7 rt=387.90 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 2850 num_peaks=40 rt=389.52 ms_level=2
INFO:TreeController:Received MS1 scan Scan 2851 num_peaks=7 rt=389.65 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.

INFO:TreeController:Received MS1 scan Scan 3293 num_peaks=6 rt=453.46 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3297 num_peaks=30 rt=453.98 ms_level=2
INFO:TreeController:Received MS2 scan Scan 3298 num_peaks=10 rt=454.11 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3304 num_peaks=6 rt=455.17 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3308 num_peaks=30 rt=455.82 ms_level=2
INFO:TreeController:Received MS2 scan Scan 3309 num_peaks=10 rt=455.97 ms_level=2
INFO:TreeController:Received M

INFO:TreeController:Received MS2 scan Scan 3522 num_peaks=40 rt=490.79 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3797 num_peaks=6 rt=526.86 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3805 num_peaks=10 rt=528.06 ms_level=2
INFO:TreeController:Received MS2 scan Scan 3806 num_peaks=30 rt=528.19 ms_level=2
INFO:TreeController:Received MS1 scan Scan 3808 num_peaks=6 rt=528.75 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 3816 num_peaks=10 rt=530.10 ms_level=2
INFO:TreeController:Received M

DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 4058 num_peaks=40 rt=570.13 ms_level=2
INFO:TreeController:Received MS1 scan Scan 4065 num_peaks=6 rt=571.33 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0)]], [[(800.0, 900.0)]], [[(900.0, 1010.0)]]]
INFO:TreeController:Received MS2 scan Scan 4069 num_peaks=40 rt=572.16 ms_level=2
INFO:TreeController:Received MS1 scan Scan 4076 num_peaks=6 rt=573.22 ms_level=1
DEBUG:TreeController:Window locations [[[(-10.0, 100.0)]], [[(100.0, 200.0)]], [[(200.0, 300.0)]], [[(300.0, 400.0)]], [[(400.0, 500.0)]], [[(500.0, 600.0)]], [[(600.0, 700.0)]], [[(700.0, 800.0